In [2]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import os
import numpy as np
import pygsp

num_classes = 10
input_shape = (28, 28, 1)

# Get the working directory path
current_dir = os.getcwd()

# Load the MNIST dataset and normalize the pixel values
(x_train, y_train), (x_test, y_test) = mnist.load_data(path=current_dir + '/mnist.zip')
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Define a simple graph filtering function
def graph_filtering(signal, laplacian):
    dense_laplacian = laplacian.toarray()  # Convert to dense matrix
    filtered_signal = np.dot(dense_laplacian, signal).flatten()  # Perform graph filtering
    return filtered_signal

# Create a 1D chain graph manually
N = x_train.shape[1] * x_train.shape[2] 
adjacency = np.eye(N, k=1) + np.eye(N, k=-1)
G = pygsp.graphs.Graph(adjacency)

# Compute the Laplacian matrix of the graph
G.compute_laplacian("normalized")

# Perform graph filtering on the training data
x_train_flattened = x_train.reshape(-1, N) # Flattened image size
x_train_filtered = np.zeros_like(x_train_flattened)
for i in range(len(x_train)):
    x_train_filtered[i, :] = graph_filtering(x_train_flattened[i, :], G.L)

# Perform graph filtering on the test data
x_test_flattened = x_test.reshape(-1, N)
x_test_filtered = np.zeros_like(x_test_flattened)
for i in range(len(x_test)):
    x_test_filtered[i, :] = graph_filtering(x_test_flattened[i, :], G.L)

# Create the model with the flattened input shape
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(N,)))  # Flattened input shape
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile and train the model with the filtered data
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train_filtered, y_train, epochs=20, batch_size=128, validation_data=(x_test_filtered, y_test))

# Evaluate the model on the filtered test data
test_accuracy = model.evaluate(x_test_filtered, y_test)
print(f'Test accuracy with graph filtering: {test_accuracy}')

Epoch 1/20
469/469 [==============================] - 1s 2ms/step - loss: 1.5006 - accuracy: 0.4762 - val_loss: 0.7812 - val_accuracy: 0.7724
Epoch 2/20
469/469 [==============================] - 1s 2ms/step - loss: 0.8739 - accuracy: 0.7085 - val_loss: 0.5031 - val_accuracy: 0.8567
Epoch 3/20
469/469 [==============================] - 1s 2ms/step - loss: 0.6817 - accuracy: 0.7817 - val_loss: 0.3866 - val_accuracy: 0.8865
Epoch 4/20
469/469 [==============================] - 1s 2ms/step - loss: 0.5761 - accuracy: 0.8181 - val_loss: 0.3288 - val_accuracy: 0.9028
Epoch 5/20
469/469 [==============================] - 1s 2ms/step - loss: 0.5149 - accuracy: 0.8398 - val_loss: 0.2920 - val_accuracy: 0.9118
Epoch 6/20
469/469 [==============================] - 1s 2ms/step - loss: 0.4690 - accuracy: 0.8536 - val_loss: 0.2699 - val_accuracy: 0.9191
Epoch 7/20
469/469 [==============================] - 1s 2ms/step - loss: 0.4377 - accuracy: 0.8639 - val_loss: 0.2571 - val_accuracy: 0.9244
Epoch 

In [3]:
x_train_flattened_size = x_train_flattened.shape
print(f"Size of x_train_flattened: {x_train_flattened_size}")

print("Matrix representation of flattened images:")
for i in range(1):  # Print the first 10 images for demonstration
    print(x_train_flattened[i])

Size of x_train_flattened: (60000, 784)
Matrix representation of flattened images:
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.       

In [4]:
laplacian_size = G.L.shape
print(f"Size of Laplacian matrix: {laplacian_size}")
print("Laplacian matrix:")
print(G.L)

Size of Laplacian matrix: (784, 784)
Laplacian matrix:
  (0, 0)	1.0
  (0, 1)	-0.7071067811865476
  (1, 0)	-0.7071067811865476
  (1, 1)	1.0
  (1, 2)	-0.5000000000000001
  (2, 1)	-0.5000000000000001
  (2, 2)	1.0
  (2, 3)	-0.5000000000000001
  (3, 2)	-0.5000000000000001
  (3, 3)	1.0
  (3, 4)	-0.5000000000000001
  (4, 3)	-0.5000000000000001
  (4, 4)	1.0
  (4, 5)	-0.5000000000000001
  (5, 4)	-0.5000000000000001
  (5, 5)	1.0
  (5, 6)	-0.5000000000000001
  (6, 5)	-0.5000000000000001
  (6, 6)	1.0
  (6, 7)	-0.5000000000000001
  (7, 6)	-0.5000000000000001
  (7, 7)	1.0
  (7, 8)	-0.5000000000000001
  (8, 7)	-0.5000000000000001
  (8, 8)	1.0
  :	:
  (775, 775)	1.0
  (775, 776)	-0.5000000000000001
  (776, 775)	-0.5000000000000001
  (776, 776)	1.0
  (776, 777)	-0.5000000000000001
  (777, 776)	-0.5000000000000001
  (777, 777)	1.0
  (777, 778)	-0.5000000000000001
  (778, 777)	-0.5000000000000001
  (778, 778)	1.0
  (778, 779)	-0.5000000000000001
  (779, 778)	-0.5000000000000001
  (779, 779)	1.0
  (779, 7